In [1]:
#%pip install torch --index-url https://download.pytorch.org/whl/cpu
%pip install datasets python-dotenv

from utils import *


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


True

In [26]:

import random
from datasets import load_dataset, Dataset
import json
from openai import OpenAI
from openai.types.chat.chat_completion import ChatCompletion
import os

domain_events_schema = {
    "type": "object",
    "properties": {
        "domain_events": {
            "type": "array",
            "items": {
                "type": "object",
                "properties": {
                    "name": {
                        "type": "string",
                        "description": "The name of the domain event in past tense, e.g. 'User created' or 'Order placed'",
                    },
                    "description": {
                        "type": "string",
                        "description": "The description of the domain event, when it occurs and what it represents",
                    },
                    "severity": {
                        "type": "string",
                        "enum": ["low", "medium", "high"],
                        "description": "The severity of the domain event. Low if it is not critical, medium if it is important but not critical, high if it is critical",
                    },
                    "schema": {
                        "type": "object",
                        "description": "The detailed schema of the domain events as valid json-schema. It contains all the metadata of the domain event, such as name of relevant entities, meassurements, findings, codes, timestamps, notes, etc.",
                        "properties": {
                            "type": {
                                "type": "string"
                            },
                            "description": {
                                "type": "string"
                            },
                            "properties": {
                                "type": "object",
                            },
                        },
                        "additionalProperties": True,
                        "required": ["type", "properties"]
                    }
                },
                "required": ["name", "description", "schema"]
            },
            "minItems": 10,
        },
    }
}

def generate_domain_events(entry):
    results = []

    available_models = [
        "mistralai/mistral-nemo",
        #"mistralai/mixtral-8x7b-instruct",
        "mistralai/mixtral-8x22b-instruct",
        "mistralai/mixtral-8x22b-instruct",
        "mistralai/mixtral-8x22b-instruct",
        "mistralai/mixtral-8x22b-instruct",
        "mistralai/mistral-small",
        #"mistralai/mistral-medium",
        "mistralai/mistral-large"
    ]

    model = random.choice(available_models)

    openrouter_response: ChatCompletion = openrouter.chat.completions.create(
        model=model,
        messages=[{
            "role": "system",
            "content": f"You are an expert {entry['profession']}. A domain event is a significant occurrence in a specific domain that is of interest to stakeholders and that can be described in terms of a set of attributes. Domain events are used to communicate changes in the state of the domain and to trigger actions in response to those changes."
        }, {
            "role": "user",
            "content": f"Generate a list of 10 domain event for the profession {entry['profession']} in the process {entry['process']}. Use the tool call generate_domain_events to generate the domain events."
        }],
        tools=[{
            "type": "function",
            "function": {
                "name": "generate_domain_events",
                "parameters": domain_events_schema,
            },
        }],
        tool_choice="required",
        max_tokens=8191,
        temperature=1.2,
        extra_body={
            "provider": {
                "require_parameters": True,
            },
        }
    )

    domain_events = json.loads(openrouter_response.choices[0].message.tool_calls[0].function.arguments)

    for domain_event in domain_events["domain_events"]:
        results.append({
            "profession": entry["profession"],
            "process": entry["process"],
            "name": domain_event["name"],
            "description": domain_event["description"],
            "severity": domain_event["severity"],
            "schema": domain_event["schema"],
        })
    
    return results


processes = load_dataset("lucaelin/generic_processes_v1", split="train")
domain_events = processes.shuffle().map(
    sane_batch_map(generate_domain_events, {"profession": [], "process": [], "name": [], "description": [], "severity": [], "schema": []}), 
    batched=True, 
    batch_size=4, 
    num_proc=6, 
    remove_columns=processes.column_names
)
#domain_events.sort(["profession", "process", "name"]).to_list()
domain_events.sort(["profession", "process", "name"]).push_to_hub("lucaelin/generic_domain_events_v1")

Map (num_proc=4):   0%|          | 0/1000 [00:00<?, ? examples/s]

{'profession': 'Dental Hygienist', 'process': 'Sterilization of Dental Instruments'}
Traceback (most recent call last):
  File "/tmp/ipykernel_417895/2306041828.py", line 22, in batch_map
    results = results + fn(example)
                        ^^^^^^^^^^^
  File "/tmp/ipykernel_417895/3569200155.py", line 104, in generate_domain_events
    domain_events = json.loads(openrouter_response.choices[0].message.tool_calls[0].function.arguments)
                               ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~^^^
TypeError: 'NoneType' object is not subscriptable



Map (num_proc=4):   2%|▏         | 20/1000 [02:00<1:26:17,  5.28s/ examples]

{'profession': 'Maid & Housekeeper', 'process': 'Grocery Shopping'}
Traceback (most recent call last):
  File "/tmp/ipykernel_417895/2306041828.py", line 22, in batch_map
    results = results + fn(example)
                        ^^^^^^^^^^^
  File "/tmp/ipykernel_417895/3569200155.py", line 104, in generate_domain_events
    domain_events = json.loads(openrouter_response.choices[0].message.tool_calls[0].function.arguments)
                               ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~^^^
TypeError: 'NoneType' object is not subscriptable



Map (num_proc=4):   5%|▍         | 48/1000 [04:07<1:03:10,  3.98s/ examples]

...

{'profession': 'Computer Systems Analyst', 'process': 'System Integration'}
Traceback (most recent call last):
  File "/tmp/ipykernel_417895/2306041828.py", line 22, in batch_map
    results = results + fn(example)
                        ^^^^^^^^^^^
  File "/tmp/ipykernel_417895/3569200155.py", line 104, in generate_domain_events
    domain_events = json.loads(openrouter_response.choices[0].message.tool_calls[0].function.arguments)
                               ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~^^^
TypeError: 'NoneType' object is not subscriptable



Map (num_proc=4):  99%|█████████▉| 994/1000 [1:47:38<01:13, 12.30s/ examples]

{'profession': 'Insurance Agent', 'process': 'Policy Review'}
Traceback (most recent call last):
  File "/tmp/ipykernel_417895/2306041828.py", line 22, in batch_map
    results = results + fn(example)
                        ^^^^^^^^^^^
  File "/tmp/ipykernel_417895/3569200155.py", line 110, in generate_domain_events
    "name": domain_event["name"],
            ~~~~~~~~~~~~^^^^^^^^
TypeError: string indices must be integers, not 'str'



Uploading the dataset shards: 100%|██████████| 1/1 [00:01<00:00,  1.74s/it]


CommitInfo(commit_url='https://huggingface.co/datasets/lucaelin/generic_domain_events_v1/commit/4bef7994b2e279b048e0210e54b8cd27a9d6be1e', commit_message='Upload dataset', commit_description='', oid='4bef7994b2e279b048e0210e54b8cd27a9d6be1e', pr_url=None, pr_revision=None, pr_num=None)